In [1]:
import math
import numpy as np

In [2]:
# Given values
sigma = 0.28
S0 = 37
K = 40
T = 0.75
r = 0.04
q = 0.02
alpha_temp = 5
M = 64

P_amer_bin = 5.0455539623

# Computing tau_final and the bounds for x
tau_final = (T * (sigma**2)) / 2
x_left = (math.log(S0/K)) + ((r - q - ((sigma**2)/2))*T) - (3*sigma*math.sqrt(T))
x_right = (math.log(S0/K)) + ((r - q - ((sigma**2)/2))*T) + (3*sigma*math.sqrt(T))

# Finite difference discretization
delta_tau =( tau_final / M)
N = math.floor((x_right - x_left) / math.sqrt(delta_tau / alpha_temp))
delta_x = (x_right - x_left) / N
alpha = delta_tau / (delta_x**2)

print(f"tau_final: {tau_final}, x_left: {x_left}, x_right: {x_right}")
print(f"delta_tau: {delta_tau}, N: {N}, delta_x: {delta_x}, alpha: {alpha}")

# Variables for transformation
a = ((r - q) / (sigma**2)) - 0.5
b = (((r - q) / (sigma**2)) + 0.5)**2 + ((2*q)/(sigma**2))

x_compute = math.log(S0/K)

tau_final: 0.029400000000000003, x_left: -0.8198228806486403, x_right: 0.6350997977092168
delta_tau: 0.00045937500000000004, N: 151, delta_x: 0.009635249525548723, alpha: 4.9481336805555545


In [3]:
def f(x):
    return K * math.exp(a*x) * max(1 - math.exp(x), 0)

def g_left(tau):
    return K * math.exp((a*x_left) + (b*tau)) * (1 - math.exp(x_left))

def g_right(tau):
    return 0


In [4]:
def forward_subst(L,b):
    
    N = len(L)
    
    x = np.zeros(N)
    
    x[0] = b[0]/L[0][0]
    
    for j in range(1,N):
        row_sum = 0
        for k in range(j):
            row_sum += L[j][k]*x[k]
        x[j] = (b[j] - row_sum)/L[j][j]
    
    return x

In [5]:
def SOR(A , b , x0 , t1 , t2 , tol1 , tol2 , w  ):
    
    N = len(A)
    
    L_A = np.tril(A)
    U_A = np.triu(A)
    
    D_A = np.zeros((N,N))
    
    for i in range(N):
        D_A[i][i] = A[i][i]
        L_A[i][i] = 0
        U_A[i][i] = 0
    
    c = w*forward_subst(D_A + (w*L_A) , b)
    
    x1 = np.full(N,math.inf)
    
    xprev = x0
    x_new = np.full(N,math.inf)
    _iter = 0
    #residual = np.linalg.norm(b - np.dot(A,x0))
    #consec = np.linalg.norm(x1 - x0)
    if t1 and t2:
        while np.linalg.norm(x_new - x0) > tol1 or np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    elif t1:
        while np.linalg.norm(x_new - x0) > tol1: #or np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    elif t2:
        while np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    print(_iter) 
    return x_new

In [6]:
ee = [[0 for j in range(N+1)] for i in range(M+1)]

for m in range(1 , M+1):
    for i in range(1, N-1 +1):
        t = x_left + (i*delta_x)
        ttau = m*delta_tau
        
        ee[m][i] = K * math.exp(a*t + b*ttau) * max(1 - math.exp(t) , 0)

In [7]:
import pandas as pd
pd.DataFrame(ee)

,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,150,151
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0,27.096591,26.823067,26.548656,26.273351,25.997140,25.720016,25.441969,25.162990,24.883068,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0,27.110042,26.836382,26.561836,26.286393,26.010046,25.732784,25.454599,25.175481,24.895421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0,27.123500,26.849705,26.575022,26.299443,26.022958,25.745559,25.467235,25.187979,24.907780,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0,27.136965,26.863034,26.588214,26.312498,26.035876,25.758339,25.479878,25.200483,24.920144,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0,27.901758,27.620106,27.337541,27.054055,26.769637,26.484278,26.197969,25.910700,25.622461,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
61,0,27.915609,27.633817,27.351112,27.067485,26.782926,26.497426,26.210975,25.923563,25.635181,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
62,0,27.929467,27.647535,27.364690,27.080922,26.796222,26.510580,26.223986,25.936432,25.647906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
63,0,27.943332,27.661260,27.378275,27.094366,26.809524,26.523740,26.237005,25.949307,25.660639,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [8]:
def crank_nicolson(M):
    
    delta_tau = tau_final / M
    N = math.floor((x_right - x_left) / math.sqrt(delta_tau / alpha_temp))
    delta_x = (x_right - x_left) / N
    alpha = delta_tau / (delta_x ** 2)
    
    print('M =', M)
    print('N =', N)
    print('Delta_tau =', delta_tau)
    print('Delta_x =', delta_x)
    print('Alpha =', alpha)
    
    U = [[0 for j in range(N+1)] for i in range(M+1)]

    # Initial condition
    for j in range(N+1):
        U[0][j] = f(x_left + j*delta_x)
    
    
    # Left Boundary condtion
    for i in range(1 , M+1):
        U[i][0] = g_left(i*delta_tau)
    
    #Right Boundary Condtion
    for i in range(1 , M+1):
        U[i][N] = g_right(i*delta_tau)
        
    # Boundary conditions are handled during the implicit step
    # Create the tridiagonal matrix and the solution vector
    A = np.zeros((N-1, N-1))
    B = np.zeros(N-1)

    # Main loop for the Crank-Nicolson scheme
    for i in range(0, M):

        # Set up the tridiagonal system
        for j in range(1, N):
            if j > 1:
                A[j-1, j-2] = -0.5 * alpha
            A[j-1, j-1] = 1 + alpha
            if j < N - 1:
                A[j-1, j] = -0.5 * alpha
            
            B[j-1] = (0.5 * alpha * U[i][j-1]) + ((1 - alpha) * U[i][j]) + (0.5 * alpha * U[i][j+1])
        
        B[0] += (alpha/2)*U[i+1][0]
        B[-1] += (alpha/2)*U[i+1][N]
        
        # Solve the tridiagonal system
        U_next = SOR(A,B, U[i][1:N],True , False, 1e-6 , 1e-6 , 1.2)
        
        # Addition for early exercise 
        
        # Update the solution
        for j in range(1, N):
            U[i+1][j] = max(U_next[j-1],ee[i+1][j])

        # Boundary conditions
        #U[i+1][0] = g_left((i+1) * delta_tau)
        #U[i+1][N] = g_right((i+1) * delta_tau)

    return U

U = crank_nicolson(M)


M = 64
N = 151
Delta_tau = 0.00045937500000000004
Delta_x = 0.009635249525548723
Alpha = 4.9481336805555545
25
25
25
25
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23


In [9]:
def g_left_ep(tau):
    return K * math.exp(a*x_left + b*tau) * (math.exp(-2*r*tau/sigma**2) - math.exp(x_left - 2*q*tau/sigma**2))

In [ ]:
def crank_nicolson_ep(M):
    
    delta_tau = tau_final / M
    N = math.floor((x_right - x_left) / math.sqrt(delta_tau / alpha_temp))
    delta_x = (x_right - x_left) / N
    alpha = delta_tau / (delta_x ** 2)
    
    print('M =', M)
    print('N =', N)
    print('Delta_tau =', delta_tau)
    print('Delta_x =', delta_x)
    print('Alpha =', alpha)
    
    U = [[0 for j in range(N+1)] for i in range(M+1)]

    # Initial condition
    for j in range(N+1):
        U[0][j] = f(x_left + j*delta_x)
    
    
    # Left Boundary condtion
    for i in range(1 , M+1):
        U[i][0] = g_left_ep(i*delta_tau)
    
    #Right Boundary Condtion
    for i in range(1 , M+1):
        U[i][N] = g_right(i*delta_tau)
        
    # Boundary conditions are handled during the implicit step
    # Create the tridiagonal matrix and the solution vector
    A = np.zeros((N-1, N-1))
    B = np.zeros(N-1)

    # Main loop for the Crank-Nicolson scheme
    for i in range(0, M):

        # Set up the tridiagonal system
        for j in range(1, N):
            if j > 1:
                A[j-1, j-2] = -0.5 * alpha
            A[j-1, j-1] = 1 + alpha
            if j < N - 1:
                A[j-1, j] = -0.5 * alpha
            
            B[j-1] = (0.5 * alpha * U[i][j-1]) + ((1 - alpha) * U[i][j]) + (0.5 * alpha * U[i][j+1])
        
        B[0] += (alpha/2)*U[i+1][0]
        B[-1] += (alpha/2)*U[i+1][N]
        
        # Solve the tridiagonal system
        U_next = SOR(A,B, U[i][1:N],True , False, 1e-6 , 1e-6 , 1.2)
        
        # Addition for early exercise 
        
        # Update the solution
        for j in range(1, N):
            U[i+1][j] = U_next[j-1]

        # Boundary conditions
        #U[i+1][0] = g_left((i+1) * delta_tau)
        #U[i+1][N] = g_right((i+1) * delta_tau)

    return U

U_ep = crank_nicolson(M)

M = 64
N = 151
Delta_tau = 0.00045937500000000004
Delta_x = 0.009635249525548723
Alpha = 4.9481336805555545
25
25
25
25
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24


In [ ]:
import pandas as pd
pd.DataFrame(U)

In [ ]:
from scipy.stats import norm

def black_scholes_put(S0, K, r, q, T, sigma):
    d1 = (math.log(S0/K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    put_price = K * math.exp(-r * T) * norm.cdf(-d2) - S0 * math.exp(-q * T) * norm.cdf(-d1)
    return put_price
    
#N = 22
def pointwise_convergence(U):
    x_compute = math.log(S0/K)
    print('x_compute = ' , x_compute)
    for i in range(N):
        if (x_left + (i*delta_x) <= x_compute)  and  (x_compute < x_left + (i+1)*delta_x):
            print(i)
            
            xi, xi1 = x_left + (i*delta_x), x_left + ((i+1)*delta_x)
            Si, Si1 = K*math.exp(xi), K*math.exp(xi1)
            print('S_left = ' , Si)
            print('S_right = ' , Si1)
            Vi, Vi1 = math.exp(-a*xi - b*tau_final)*U[M][i], math.exp(-a*xi1 - b*tau_final)*U[M][i+1]
            V_approx = (((Si1 - S0)*Vi) + ((S0 - Si)*Vi1)) / (Si1 - Si)
            V_exact = P_amer_bin
            error_pointwise = abs(V_approx - V_exact) #/ V_exact

            # Interpolation
            u_xcompute = (((xi1 - x_compute)*U[M][i]) + ((x_compute - xi)*U[M][i+1]) )/ (xi1 - xi)
            V_approx2 = math.exp(-a*x_compute - b*tau_final)*u_xcompute
            error_pointwise2 = abs(V_approx2 - V_exact) #/ V_exact

            return error_pointwise, error_pointwise2, V_approx

error_pointwise, error_pointwise2, Vapprox = pointwise_convergence(U)
error_pointwise_ep, error_pointwise2_ep, Vapprox_ep = pointwise_convergence(U_ep)
print(Vapprox)

In [ ]:
BS = black_scholes_put(S0, K, r, q, T, sigma)

In [ ]:
error_pointwise, error_pointwise2

In [ ]:
#X-grid
x = []
S = []
for j in range(N+1):
    t = x_left + j*delta_x
    x.append(t)
    S.append(K*math.exp(t))

In [ ]:
S

In [ ]:
def compute_rms_error(U, S, a, b, tau_final, K, r, q, sigma, S0, threshold=0.00001):
    
    V_approx = [math.exp(-(a * x[i]) - (b * tau_final)) * U[-1][i] for i in range(len(x))]
    V_exact = [black_scholes_put(S[i], K, r, q,T , sigma) for i in range(len(S))]
    
    valid_indices = [i for i, val in enumerate(V_exact) if val > threshold * S0]
    NRMS = len(valid_indices)
    
    print(valid_indices )
    print(V_exact)
    errors = [((V_approx[i] - V_exact[i])**2) /(V_exact[i]**2) for i in valid_indices]
    rms_error = math.sqrt(sum(errors) / NRMS)
    return rms_error

In [ ]:
def compute_greeks(U, S, a, b, tau_final, delta_tau, delta_x, i):
    # Values of option at nodes of interest
    Vi = math.exp(-a * x[i] - b * tau_final) * U[-1][i]
    Vi_plus_1 = math.exp(-a * x[i+1] - b * tau_final) * U[-1][i+1]
    Vi_minus_1 = math.exp(-a * x[i-1] - b * tau_final) * U[-1][i-1]
    Vi_plus_2 = math.exp(-a * x[i+2] - b * tau_final) * U[-1][i+2]
    Vi_delta_t = math.exp(-a * x[i] - b * (tau_final - delta_tau)) * U[-2][i]
    Vi1_delta_t = math.exp(-a * x[i+1] - b * (tau_final - delta_tau)) * U[-2][i+1]
    
    # Delta
    delta_fd = (Vi_plus_1 - Vi) / (S[i+1] - S[i])
    
    # Gamma
    gamma_fd = ((Vi_plus_2 - Vi_plus_1) / (S[i+2] - S[i+1])) - ((Vi - Vi_minus_1) / (S[i] - S[i-1])) 
    gamma_fd /= (S[i+2] + S[i+1])/2 - (S[i] + S[i-1])/2
    
    # Theta
    V_approx_current = (((S[i+1] - S0) * Vi) + ((S0 - S[i]) * Vi_plus_1)) / (S[i+1] - S[i])
    
    V_approx_previous = (((S[i+1] - S0) * Vi_delta_t) + ((S0 - S[i]) * Vi1_delta_t)) / (S[i+1] - S[i])
    
    delta_t = (2 * delta_tau) / (sigma**2)
    theta_fd = (V_approx_previous - V_approx_current) / delta_t

    return delta_fd, gamma_fd, theta_fd


In [ ]:
rms_error = compute_rms_error(U, S, a, b, tau_final, K, r, q, sigma, S0)
print(f"RMS Error: {rms_error}")

# Find i such that xi <= xcompute < xi+1
i = (next(idx for idx, val in enumerate(x) if val > x_compute)) - 1
delta_fd, gamma_fd, theta_fd = compute_greeks(U, S, a, b, tau_final, delta_tau, delta_x, i)

print('I = ' , i )

print(f"Delta (Δ): {delta_fd}")
print(f"Gamma (Γ): {gamma_fd}")
print(f"Theta (Θ): {theta_fd}")


In [ ]:
print("{:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f}".format(error_pointwise,  error_pointwise2,   rms_error, delta_fd, gamma_fd, theta_fd))


In [ ]:
# Round values to maintain 6 digits after the decimal
error_pointwise = round(error_pointwise, 6)
error_pointwise2 = round(error_pointwise2, 6)
rms_error = round(rms_error, 6)
delta_fd = round(delta_fd, 6)
gamma_fd = round(gamma_fd, 6)
theta_fd = round(theta_fd, 6)

# Create a DataFrame
data = {
    'error_pointwise': [error_pointwise],
    'gap': [''],  # gap
    'error_pointwise2': [error_pointwise2],
    'gap2': [''],  # gap
    'delta_fd': [delta_fd],
    'gamma_fd': [gamma_fd],
    'theta_fd': [theta_fd],
    'var_red': [Vapprox + (BS - Vapprox_ep)],
    'var_red_pointwise': [abs(Vapprox + (BS - Vapprox_ep) - P_amer_bin)]
}
df = pd.DataFrame(data)

# Display the DataFrame
display(df)